In [1]:
import os
from os import path
import pandas as pd
import sys
basepath = path.join(os.getcwd(), "export_us")

In [2]:
import os
import numpy as np
import pandas as pd

start, end = '2020-06-30', '2024-06-27'
stock_codes = pd.read_csv('sp500_companies.csv')['Symbol']

dfs = []
for stock_code in stock_codes:
    fname = f"{stock_code}.csv"
    fpath = os.path.join(basepath, fname)
    if os.path.exists(fpath):
        df = pd.read_csv(fpath, parse_dates=['日期'])
        df.rename(columns={'日期': 'date', '开盘': 'open', '收盘': 'close', '最高': 'high', '最低': 'low', '成交量': 'volume', '成交额': 'amount'}, inplace=True)
        df['stock_code'] = stock_code
        dfs.append(df[(df['date'] >= start) & (df['date'] <= end)])

all_df = pd.concat(dfs, ignore_index=True)
all_df.set_index(['stock_code', 'date'], inplace=True)
all_df.sort_index(inplace=True)

/tmp/ipykernel_2448358/1281393525.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_df = pd.concat(dfs, ignore_index=True)


In [3]:
all_df

open    close     high      low    volume  \
stock_code date                                                       
AAPL       2020-06-30   85.385   86.565   86.860   85.365  35055821   
           2020-07-01   86.645   86.393   87.205   86.343  27684309   
           2020-07-02   87.328   86.393   87.983   86.275  28510367   
           2020-07-06   87.865   88.828   89.310   87.833  29663913   
           2020-07-07   89.218   88.538   90.020   88.423  28106114   
...                        ...      ...      ...      ...       ...   
ZBRA       2024-02-08  247.220  247.120  249.745  245.680    318724   
           2024-02-09  248.360  253.090  253.770  248.360    328612   
           2024-02-12  253.090  259.400  260.000  253.090    395803   
           2024-02-13  251.780  245.800  253.105  243.485    692213   
           2024-02-14  249.470  252.960  255.355  248.270    567311   

                             amount    振幅   涨跌幅     涨跌额   换手率  
stock_code date                                                
AAPL       2020-06-30  1.275552e+10  1.74  0.88   0.755  0.82  
           2020-07-01  1.012571e+10  1.00 -0.20  -0.172  0.65  
           2020-07-02  1.046759e+10  1.98  0.00   0.000  0.67  
           2020-07-06  1.107689e+10  1.71  2.82   2.435  0.69  
           2020-07-07  1.056394e+10  1.80 -0.33  -0.290  0.66  
...                             ...   ...   ...     ...   ...  
ZBRA       2024-02-08  7.882456e+07  1.65  0.00   0.010  0.62  
           2024-02-09  8.274616e+07  2.19  2.42   5.970  0.64  
           2024-02-12  1.024864e+08  2.73  2.49   6.310  0.77  
           2024-02-13  1.711448e+08  3.71 -5.24 -13.600  1.35  
           2024-02-14  1.430427e+08  2.88  2.91   7.160  1.10  

[135571 rows x 10 columns]

In [4]:
all_df.to_csv("all_stock_data.csv")

In [5]:
all_df_loaded = pd.read_csv("all_stock_data.csv", parse_dates=['date'])
all_df_loaded.set_index(['stock_code', 'date'], inplace=True)
all_df_loaded.sort_index(inplace=True)

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
from scipy.stats import pearsonr


# 读取合并后的数据
all_df_loaded = pd.read_csv("all_stock_data.csv", parse_dates=['date'])
all_df_loaded.set_index(['stock_code', 'date'], inplace=True)
all_df_loaded.sort_index(inplace=True)


# 定义你的因子函数，输入为单个股票的DataFrame
def factor_func(df, lookback_period_vwma=20, lookback_period_momentum=10, market_regime='Neutral'):
    price_diff_sma = (df['high'] - df['low']).rolling(window=20).mean()
    volume_ema = df['volume'].ewm(span=10, adjust=False).mean()
    heuristics_matrix = price_diff_sma * volume_ema
    return heuristics_matrix

# 计算每只股票的因子值
all_df_loaded['factor'] = all_df_loaded.groupby('stock_code').apply(lambda g: factor_func(g)).reset_index(level=0, drop=True)

# 计算未来6日收益率
all_df_loaded['future_return_6d'] = all_df_loaded.groupby('stock_code')['close'].shift(-6) / all_df_loaded['close'] - 1

# 取所有日期
start_date = pd.Timestamp('2021-01-01')
end_date = pd.Timestamp('2024-01-01')
all_dates = all_df_loaded.index.get_level_values('date').unique()
all_dates = all_dates[(all_dates >= start_date) & (all_dates <= end_date)]
ic_values = []

for date in all_dates:
    daily = all_df_loaded.xs(date, level='date')
    factors = daily['factor']
    returns = daily['future_return_6d']
    mask = factors.notna() & returns.notna()
    if mask.sum() >= 10:
        ic, _ = pearsonr(factors[mask], returns[mask])
        if not np.isnan(ic):
            ic_values.append(ic)

mean_ic = np.mean(ic_values)
print(f"Mean IC: {mean_ic:.10f}")

Mean IC: -0.0091599866
